In [ ]:
import pandas as pd
import ipysheet
import nltk

from utils.nlu_engine import NLUEngine
from utils.nlu_engine import MacroDataRefinement
from utils.nlu_engine import DataUtils
from utils.nlu_engine import IntentMatcher, LR
from utils.nlu_engine import EntityExtractor


# Macro NLU Data Refinement

It's a bit like the TV show [Serverance](https://www.imdb.com/title/tt11280740/) .

![Helly R and Mark S](https://media.npr.org/assets/img/2022/02/15/atv_severance_photo_010103-5f8033cc2b219ba64fe265ce893eae4c90e83896-s1100-c50.jpg "Helly R and Mark G")

*Helly R*: `My job is to scroll through the spreadsheet and look for the numbers that feel scary?`

*Mark S*: `I told you, you’ll understand when you see it, so just be patient.`

![MDR](https://www.imore.com/sites/imore.com/files/styles/large/public/field/image/2022/03/refinement-software-severance-apple-tv.jpg "serverance micro data refinement")

*Helly R*: `That was scary. The numbers were scary.`

Hopefully the intents and entities that are wrong aren't scary, just a bit frustrating. Let's see if we can find the right ones.

* Macro NLU Data Refinement: Intent
* Macro NLU Data Refinement: Entity


In [ ]:
# TODO: Refactor with text inputs for increased flexibility like this: https://queirozf.com/entries/interactive-controls-for-jupyter-notebooks-python-examples

In [ ]:
nlu_data_df = DataUtils.load_data(
    'data/NLU-Data-Home-Domain-Annotated-All-Cleaned.csv'
)

In [ ]:
nlu_data_df

## Intent

### Create intent classifier report

Let's do a report by domain classification.

In [ ]:
domain_labels = 'scenario'

domain_report_df = NLUEngine.evaluate_intent_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict=domain_labels,
    classifier=LR
)

domain_report_df

And now let's do a report by intent classification.

In [ ]:
intent_report_df = NLUEngine.evaluate_intent_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict='intent',
    classifier=LR
)
intent_report_df

### Macro Intent Data Refinement

Now that we know what works and what doesn't, we can start refining the intents.

We don't want all of the columns, so we will drop some to review the data set

In [ ]:
nlu_scenario_df = nlu_data_df.drop(
    columns=[
        'userid', 'notes', 'answer', 'answerid', 'suggested_entities'
    ])

Pick a domain (scenario) to review

For this example we are going to pick `alarm` as an example but for actual refinement, pick whatever you like.

The intent classification isn't bad, but the entity extraction for alarm_type is terrible. Perhaps it overlaps with another entity type, like 'event_name'. We will try to fix this.

In [ ]:
#TODO: add in the lists of domains and select one (use input)

domain = 'alarm'

In [ ]:
domain_report_df[domain_report_df['domain'] == domain]

In [ ]:
nlu_scenario_df = nlu_scenario_df[
    nlu_scenario_df['scenario'] == domain
]
nlu_scenario_df

TODO: add in description of the types of fixes we can do to the NLU data for intent
* intents that collide with other intents and how to fix them (separation by TFIDF terms and using checkboxes in ipysheet to annotate them into the correct intent): this leads to the visualization of the intents in the NLU data with venn word cloud diagrams
* utterances that are grammatically incorrect or contain incorrect spelling (grammar checker in the future?)
* utterances that are straight up wrong for the intent
* utterances that actually seem contain multiple intents (this isn't supported by default)

Let's train an intent classifier on the whole data set for labeling intents and get the incorrect results for the intents on the domain we want to clean.
(why not split a training test set? Because we want to see the results of the intent classifier on the whole data set, I mean if it's still getting it wrong when it has trained on it, then perhaps there is something wrong with the utterance, tagging, overlapping intents, etc.)

In [ ]:
LR_intent_classifier_model, tfidf_vectorizer = NLUEngine.train_intent_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict='intent',
    classifier=LR
)

In [ ]:
#TODO: for every intent in the intent column, get the top 5 tfidf features and their scores
# Like this: https://stackoverflow.com/questions/34232190/scikit-learn-tfidfvectorizer-how-to-get-top-n-terms-with-highest-tf-idf-score
#TODO: Make sure to pass them to the intent refinement process for each intent by putting them in the report!

In [ ]:
incorrect_intent_predictions_df = IntentMatcher.get_incorrect_predicted_labels(
    nlu_scenario_df, LR_intent_classifier_model, tfidf_vectorizer)
incorrect_intent_predictions_df

In [ ]:
MacroDataRefinement.get_incorrect_predicted_intents_report(
    nlu_scenario_df, incorrect_intent_predictions_df, intent_report_df)

#TODO: export to an md file (and format it!), or better yet: just make it a JSON file!


In [ ]:
#TODO: find separation criteria for alarm_set and calander_set: get most popular tfidf words (and/or coef features) for each intent and assign them as the separation criteria. e.g.
# alarm -> alarm_set
# reminder -> calander_set
# is alert alarm or reminder?
# wakeup or wake up -> alarm_set
# get up -> alarm_set
# timer -> remove, it's not a timer!

#TODO: this will be later visually represented in a venn diagram with word clouds and forms the basis for refinement besides the sheets

### TODO: human in the for loop.

You have made it this far, now it's your turn to shine human!

You will provide a refinement to each incorrectly predicted intent. some of the incorrectly predicted utterances are actually fine the way they are, you may need to review the intent that is fasly being predicted...


Besides correcting the utterances(ie spelling), you can also mark an entry with the following:
* **review**: the utterance needs to be reviewed again by a human
* **move**: the utterance needs to be moved to another intent(NOTE: if you have a big data set, it might be better to just **remove** the utterance from the data set)
* **remove**: the utterance should be removed from the dataset

You can use your human ability to refine the NLU intent data by answering the following questions:

1. Does the utterance fit to the intent? -> mark as move, remove, or review

2. Is the utterance grammar or spelling wrong but(1) is fine? -> correct the utterance

3. Is this intent collidating with another intent because the scope of both intents are overlapping? -> redefine the scope of the intents(either combine them or separate their functionality better)

4. Is the intent collidating with another intent because certain keywords overlap between intents? -> redefine the keywords to split between intents or merge them together if they are similar


In [ ]:
# TODO: from here it's all just a work in progress. These 4 flows should be implemented in a human for loop pipeline with ipysheets.
# TODO: at the end of each flow, the dataframe will be appended with a column to indicate MDR was successfully completed. This way users can keep track of what they have refined.

In [ ]:
intent_refinement_dictionary = MacroDataRefinement.get_intent_dataframes_to_refine(incorrect_intent_predictions_df)

From the above intents, pick one to refine. It's probably a good idea to look at the `MacroDataRefinement.get_incorrect_predicted_intents_report` and start with the one with the most incorrect predictions.

Put that intent into the quotes below.

In [ ]:
#TODO: change to input field
to_review_sheet = MacroDataRefinement.create_sheet(intent_refinement_dictionary['alarm_query'])
to_review_sheet

It's a really good idea to convert the sheet back to a dataframe and convert it to a csv to be saved.

NOTE: make sure to replace the `domain` with the domain and `intent` in the csv file name to the actual domain and intent name you have refined in both `to_csv()` and `load_data()`.

In [ ]:
reviewed_intent_df = MacroDataRefinement.convert_sheet_to_dataframe(to_review_sheet)

In [ ]:
reviewed_intent_df.to_csv(
    'data/reviewed/reviewed_alarm_alarm_query_incorrectly_predicted_df.csv')


In [ ]:
reviewed_intent_df = pd.read_csv(
    'data/reviewed/reviewed_alarm_alarm_query_incorrectly_predicted_df.csv', sep=',', index_col=0)


In [ ]:
#TODO: move remove_entries_marked_remove to the end of the flow (so that when joining together back to the main dataset, those entries are removed there too!)
#TODO: add in list of intents above this cell so people can see if it's the right intent to pick

refined_intent_df = reviewed_intent_df.apply(
    MacroDataRefinement.move_entry, axis=1)

In [ ]:
refined_intent_df.to_csv(
    'data/refined/refined_alarm_alarm_query_incorrectly_predicted_df.csv', sep=',')


In [ ]:
refined_intent_df = pd.read_csv(
    'data/refined/refined_alarm_alarm_query_incorrectly_predicted_df.csv', sep=',', index_col=0)


In [ ]:
#TODO: this can be removed in the future, I forgot to remove these entries when I did my own refinement so I drop them for now
refined_intent_df = refined_intent_df[~refined_intent_df['status'].str.contains(
    'IRR', na=False)]


In [ ]:
#TODO: move this and the next cell into a function in the macro data refinement class

In [ ]:
refined_intent_df = refined_intent_df.assign(intent_refined=True)
refined_intent_df['intent_refined'] = refined_intent_df['intent_refined'].astype(bool)
refined_intent_df


In [ ]:
nlu_data_refined_df = nlu_data_df.merge(refined_intent_df, how='left', left_index=True, right_index=True)
nlu_data_refined_df['scenario_y'].fillna(nlu_data_refined_df['scenario_x'], inplace=True)
nlu_data_refined_df['intent_y'].fillna(nlu_data_refined_df['intent_x'], inplace=True)
nlu_data_refined_df['answer_annotation_y'].fillna(nlu_data_refined_df['answer_annotation_x'], inplace=True)
nlu_data_refined_df['status_y'].fillna(nlu_data_refined_df['status_x'], inplace=True)
nlu_data_refined_df['intent_refined'].fillna(
    False, inplace=True)
nlu_data_refined_df.drop(columns=['scenario_x', 'intent_x', 'answer_annotation_x', 'status_x', 'move'], inplace=True)
nlu_data_refined_df.rename(columns={'scenario_y': 'scenario', 'intent_y': 'intent', 'answer_annotation_y': 'answer_annotation', 'status_y': 'status'}, inplace=True)
nlu_data_refined_df

In [ ]:
removed_nlu_data_refined_df = nlu_data_refined_df[nlu_data_refined_df['remove'] != True]

LR_intent_classifier_model, tfidf_vectorizer = NLUEngine.train_intent_classifier(
    data_df_path=removed_nlu_data_refined_df,
    labels_to_predict='intent',
    classifier=LR
)

refined_incorrect_intent_predictions_df = IntentMatcher.get_incorrect_predicted_labels(
    removed_nlu_data_refined_df[removed_nlu_data_refined_df['scenario'] == 'alarm'], LR_intent_classifier_model, tfidf_vectorizer)
refined_incorrect_intent_predictions_df


In [ ]:
improved_intent_report_df = NLUEngine.evaluate_intent_classifier(
    data_df_path=nlu_data_df,
    labels_to_predict='intent',
    classifier=LR
)
improved_intent_report_df

In [ ]:
MacroDataRefinement.get_incorrect_predicted_intents_report(
    removed_nlu_data_refined_df[removed_nlu_data_refined_df['scenario'] == 'alarm'], refined_incorrect_intent_predictions_df, improved_intent_report_df)


Besides some incorrect utterances and intents, we can see that there is an overlap between the intent 'alarm_set' and the intent 'calandar_set'. This is because those two intents are not well defined and will require refinement. We will try to fix this.


In [ ]:
#TODO: integrate refined_intent_df into the main dataset and save it as nlu_data_refined_df


#TODO: export nlu_data_refined_df to a csv file and save it as NLU-Data-Home-Domain-Annotated-Refined.csv

In [ ]:
#TODO: for every intent in the predicted intent column, get the top 5 tfidf features and their scores
# Like this: https://stackoverflow.com/questions/34232190/scikit-learn-tfidfvectorizer-how-to-get-top-n-terms-with-highest-tf-idf-score
#TODO: Make sure to pass them to the intent refinement process for each intent by putting them in the report!


In [ ]:

#TODO: get the counts of the terms from the utterances that are incorrect for a specific domain (should I filter by tfidf scores?)
#TODO: Look up the most popular terms for an intent if they are red or green for that intent

In [ ]:
# For every word (feature) in the utterances, we get the coeficients for the intents.
# From the shape, we see it contains the classes and the coeficients.
coefs = LR_intent_classifier_model.coef_
coefs.shape

In [ ]:
from utils.nlu_engine import LabelEncoder

In [ ]:
# We need to get the encoded classes
classes = LR_intent_classifier_model.classes_
classes

In [ ]:
# We cant to get the actual feature names (the words)
feature_names = tfidf_vectorizer.get_feature_names()

In [ ]:
# Let's try an example with TFIDF only, this only tells us overall the TFIDF score for each word, not related to the intent
from utils.nlu_engine import TfidfEncoder

utterance = 'turn off the alarm I set'

response = TfidfEncoder.encode_vectors(
    utterance, tfidf_vectorizer)

for vector in response.nonzero()[1]:
    print(f'word: {feature_names[vector]} - ranking: {response[0, vector]}')


In [ ]:
# Let's rip out a list of tuples for the features and their coeficients for the intent
output = []
for classIndex, features in enumerate(coefs):
    for featureIndex, feature in enumerate(features):
        output.append(
            (classes[classIndex], feature_names[featureIndex], feature))
feature_rank_df = pd.DataFrame(output, columns=['class', 'feature', 'coef'])
feature_rank_df


In [ ]:
# It is a good idea to convert the classes from the encoded to a normal human form
feature_rank_df['class'] = LabelEncoder.inverse_transform(feature_rank_df['class'])

In [ ]:
# Sort the features by the absolute value of their coefficient and color them red or green
feature_rank_df["abs_value"] = feature_rank_df["coef"].apply(lambda x: abs(x))
feature_rank_df["colors"] = feature_rank_df["coef"].apply(lambda x: "green" if x > 0 else "red")
feature_rank_df = feature_rank_df.sort_values("abs_value", ascending=False)

In [ ]:
# Take a look at an example of the word 'set'
feature_rank_df[(feature_rank_df['feature'] == 'set') & (feature_rank_df['colors'] == 'red')
   ].sort_values('abs_value', ascending=False)


### Entity extraction report

The entity extraction could be greatly improved by improving the features it uses. It would be great if someone would take a look at this. Perhaps the CRF features similar to what Snips uses would be better such as Brown clustering (probably).

In [ ]:
#TODO: implement brown clustering to improve entity extraction (see entity_extractor.py)

It is important to have the NLTK tokenizer to be able to extract entities.

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
        nltk.download('punkt')

Due to this error featured in [this git issue](https://github.com/TeamHG-Memex/sklearn-crfsuite/issues/60) we have to use an older version of scikit learn (sklearn<0.24), otherwise the latest version would work. Hopefully this gets fixed one day..

In [ ]:
entity_report_df = NLUEngine.evaluate_entity_classifier(data_df=nlu_data_df)

In [ ]:
entity_report_df.sort_values(by=['f1-score'])

In [ ]:
#TODO: Benchmark the state features to find the best and the worst, remove/replace worst: add in state features like here: https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#let-s-check-what-classifier-learned
# Specifically, we want print_state_features()

As we have seen from the entity extraction report, the entity extraction is not working for the alarm_type.

In [ ]:

#TODO: move this below the intent cleaning flow
nlu_scenario_df = nlu_scenario_df[nlu_scenario_df['answer_annotation'].str.contains(
    'alarm_type')]

## Entity Convert to ipysheet and review
TODO: add in description of the types of fixes we can do to the NLU data for entity


In [ ]:

# TODO: same as above for intents but with predicted entities: report on them, break them down into a dictionary of dataframes and refine them..

For the example with 'alarm' and the alarm_type:
* We see that the alarm_type entities are really event_name(ie wake up, soccer practice) except for ID 5879, we will need to change them to event_name and remove ID 5879.
* The last one(ID 6320) is a mistake. Someone got confused with the prompt and assumed alarm is a security system. This is out of scope for the alarm domain, as the alarms are ones set on a phone or other device. We will drop this utterance.
Once you are done reviewing, you convert it back to a dataframe and check to make sure it looks okay.
Let's change all alarm_type entities to event_name.

In [ ]:

reviewed_scenario_df['answer_annotation'] = reviewed_scenario_df['answer_annotation'].str.replace(
    'alarm_type', 'event_name')
reviewed_scenario_df


Okay dokey, now we can merge this with the original data set and see if it made a difference already(well of course it did!).

In [ ]:
nlu_data_df.drop(
    reviewed_scenario_df[reviewed_scenario_df['remove'] == True].index, inplace=True)

reviewed_scenario_df = reviewed_scenario_df[~reviewed_scenario_df['remove'] == True]

nlu_data_df.loc[nlu_data_df.index.intersection(
    reviewed_scenario_df.index), 'answer_annotation'] = reviewed_scenario_df['answer_annotation']

nlu_data_df[(nlu_data_df['scenario'].str.contains('alarm')) & (nlu_data_df['answer_annotation'].str.contains(
    'event_name'))]


### Benchmark changed data set
TODO: repeat reports for the changed data set for domain and entities and compare


In [ ]:

entity_reviewed_report_df = NLUEngine.evaluate_entity_classifier(
    data_df=nlu_data_df)
entity_reviewed_report_df.sort_values(by=['f1-score'])

If you are sure it is okay, you can save it as a csv file, make sure to name it correctly(i.e. `alarm_domain_first_review.csv`)

In [ ]:
reviewed_scenario_df.to_csv('alarm_domain_first_review.csv')

Load it back up and check to make sure it looks okay. Make sure to give it the right name!


In [ ]:
audio_domain_first_review_df = pd.read_csv(
    'alarm_domain_first_review.csv', index_col=0)
audio_domain_first_review_df.tail(50)


In [ ]:
# TODO: implement the evaluate_classifier in the NLU engine to check f1 score for intents and entities in the domain vs original NLU data of domain!
# Value: benchmark!
#TODO: implement a flow for getting the domains with the lowest f1 scores by intent/domain and entities and cleaning them by the order of the lowest f1 scores
# TODO: concat all reviewed dfs and save to csv
# TODO: add benchmark for whole NLU data set before and after cleaning! (by intents and domains!)
# TODO: review the review marked entries
# TODO: add new column for notes
# TODO: change flow of review for only ones that should be reviewed, not all of the ones that have been changed (track changes by comparing against the original data set)
# TODO: do the changed utterances have to be changed in other fields too or is it just enough for the tagged utterancve field?
# TODO: add visualizations of domains, their intents, keywords in utterances, and entities to top
